In [21]:
import sqlite3
from pyspark.sql.functions import col
import pandas as pd

In [22]:
df_anomalies_gold_df = spark.read.parquet(f'PipelineStages/Gold/power_output_anomalies')


In [23]:
# Use our imported helper function to generate our summary dataframe
df_summary_statistics_gold_df = spark.read.parquet(f'PipelineStages/Gold/summary_statistics')


Create our tables to insert into

In [24]:
# Connect to SQLite database (this will create the database file if it doesn't exist)
conn = sqlite3.connect('wind_turbine_data.db')

# Create cursor object to execute SQL commands
cursor = conn.cursor()

# SQL to create table for anomaly data
anomaly_table = """
CREATE TABLE IF NOT EXISTS anomalies (
    timestamp TIMESTAMP,
    turbine_id INTEGER,
    power_output REAL,
    mean_power_output REAL,
    stddev_power_output REAL,
    lower_bound REAL,
    upper_bound REAL
);
"""

# SQL to create table for summary statistics
create_summary_stats_table = """
CREATE TABLE IF NOT EXISTS summary_statistics (
    turbine_id INTEGER,
    date DATE,
    min_power_output REAL,
    max_power_output REAL,
    avg_power_output REAL
);
"""

# Execute SQL commands
cursor.execute(anomaly_table)
cursor.execute(create_summary_stats_table)

# Commit changes and close connection
conn.commit()
conn.close()


Load our data into the created tables

In [25]:
# Convert timestamp columns to string in PySpark due to datatype differences (we wouldn't typically use pandas but pandas connects to SQLite whereas spark doesn't)
df_anomalies_gold_df = df_anomalies_gold_df.withColumn("timestamp", col("timestamp").cast("string"))
df_summary_statistics_gold_df = df_summary_statistics_gold_df.withColumn("date", col("date").cast("string"))

# Convert to Pandas DataFrame
anomalies_pandas_df = df_anomalies_gold_df.toPandas()
statistics_pandas_df = df_summary_statistics_gold_df.toPandas()

# Convert string columns back to datetime in Pandas
anomalies_pandas_df['timestamp'] = pd.to_datetime(anomalies_pandas_df['timestamp'])
statistics_pandas_df['date'] = pd.to_datetime(statistics_pandas_df['date'])

# Continue with writing to SQLite
conn = sqlite3.connect('wind_turbine_data.db')
anomalies_pandas_df.to_sql('anomalies', conn, if_exists='replace', index=False)
statistics_pandas_df.to_sql('summary_statistics', conn, if_exists='replace', index=False)
conn.close()


In [27]:
print("Database insert completed successfully")

Database insert completed successfully
